# TACO: Compute metrics

In [ ]:
import polars as pl
import torch
import numpy as np
import json
from datasets import load_from_disk
import ast
import signal

/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
results = json.loads(open("results.json").read())
results

[{'task_id': 4,
  'prompt': "\nQUESTION: \nYou have a deck of $n$ cards, and you'd like to reorder it to a new one.\n\nEach card has a value between $1$ and $n$ equal to $p_i$. All $p_i$ are pairwise distinct. Cards in a deck are numbered from bottom to top, i. e. $p_1$ stands for the bottom card, $p_n$ is the top card.\n\nIn each step you pick some integer $k > 0$, take the top $k$ cards from the original deck and place them, in the order they are now, on top of the new deck. You perform this operation until the original deck is empty. (Refer to the notes section for the better understanding.)\n\nLet's define an order of a deck as $\\sum\\limits_{i = 1}^{n}{n^{n - i} \\cdot p_i}$.\n\nGiven the original deck, output the deck with maximum possible order you can make using the operation above.\n\n\n-----Input-----\n\nThe first line contains a single integer $t$ ($1 \\le t \\le 1000$) — the number of test cases.\n\nThe first line of each test case contains the single integer $n$ ($1 \\le 

In [3]:
train_tests = pl.read_ipc(f"../../data/TACO/processed/train_evaluation_tests.feather")


## Build ast 

In [4]:
from unittest.mock import patch

In [5]:
list_input = train_tests.filter(pl.col("id") == 4).select(pl.col("input")).to_numpy().squeeze(1)[0].split("\n")[0:-2]


In [6]:
train_tests.filter(pl.col("id") == 4).select(pl.col("input")).to_numpy().squeeze(1)[0].split("\n")[0:-1]

['4', '4', '1 2 3 4', '5', '1 5 2 4 3', '6', '4 2 5 3 6 1', '1', '1']

In [14]:
print(results[0]["output"][2])

import sys

def max_order(n, a):
    a = [-1] + a
    b = [0] * (n + 1)
    for i in range(1, n + 1):
        b[i] = b[i - 1] + (n - i + 1) * a[i]
    c = [0] * (n + 1)
    c[0] = 0
    for i in range(1, n + 1):
        if a[i] > a[i - 1]:
            c[i] = c[i - 1] + 1
        else:
            c[i] = c[i - 1]
    d = [0] * (n + 1)
    d[0] = 0
    for i in range(1, n + 1):
        if a[i] > a[i - 1]:
            d[i] = d[i - 1] + 1
        else:
            d[i] = d[i - 1]
    for i in range(n, 0, -1):
        if c[i] == i:
            for j in range(1, n + 1 - i):
                if d[n + 1 - i + j] == n - i + j:
                    for k in range(i, n + 1, j):
                        sys.stdout.write(str(a[k]) +'')
                    for k in range(n + 1 - j, n + 1):
                        sys.stdout.write(str(a[k]) +'')
                    sys.stdout.write('\n')
                    return
        else:
            continue

t = int(input())
for _ in range(t):
    n = int(input(

{'body': [<ast.Import object at 0x7f2e5410bf70>, <ast.Assign object at 0x7f2e5410bf10>, <ast.FunctionDef object at 0x7f2e5410bb50>, <ast.FunctionDef object at 0x7f2e5410ba00>, <ast.Assign object at 0x7f2e5410b820>, <ast.For object at 0x7f2e5410b760>], 'type_ignores': []}


In [9]:
results[0]["output"]

['t = int(input())\n\nfor _ in range(t):\n    n = int(input())\n    cards = list(map(int, input().split()))\n    max_card = max(cards)\n    indices = [i for i, x in enumerate(cards) if x == max_card]\n    print(*cards[indices[-1]:] + cards[:indices[-1]])',
 'import sys\ninput = sys.stdin.readline\n\ndef read_int():\n    return int(input())\n\ndef read_ints():\n    return map(int, input().split())\n\nt = read_int()\nfor _ in range(t):\n    n = read_int()\n    p = list(read_ints())\n    p = [-x for x in p]\n    stack = []\n    for i in range(n):\n        while stack and stack[-1] > p[i]:\n            stack.pop()\n        stack.append(p[i])\n    print(*[abs(x) for x in stack])',
 "import sys\n\ndef max_order(n, a):\n    a = [-1] + a\n    b = [0] * (n + 1)\n    for i in range(1, n + 1):\n        b[i] = b[i - 1] + (n - i + 1) * a[i]\n    c = [0] * (n + 1)\n    c[0] = 0\n    for i in range(1, n + 1):\n        if a[i] > a[i - 1]:\n            c[i] = c[i - 1] + 1\n        else:\n            c[

In [16]:
exec(compiled_code)

KeyboardInterrupt: 

In [15]:
code_str = results[0]["output"][2]
parsed_code = ast.parse(code_str)
input_values = train_tests.filter(pl.col("id") == 4).select(pl.col("input")).to_numpy().squeeze(1)[0].split("\n")[0:-1]
compiled_code = compile(parsed_code, filename="<string>", mode="exec")
with patch("builtins.input", side_effect=input_values):
    exec(compiled_code)


KeyboardInterrupt: 

In [ ]:
train_tests.filter(pl.col("id") == 4).select(pl.col("output")).to_numpy().squeeze(1)[0]

'4 3 2 1\n5 2 4 3 1\n6 1 5 3 4 2\n1\n'

In [ ]:
train_tests.filter(pl.col("id") == 4).select(pl.col("input")).to_numpy().squeeze(1)[0].split("\n")[0:-2]

['4', '4', '1 2 3 4', '5', '1 5 2 4 3', '6', '4 2 5 3 6 1', '1']

In [ ]:
train_tests.filter(pl.col("id") == 4).select(pl.col("input"))[0].to_numpy().squeeze(1)

array(['4\n4\n1 2 3 4\n5\n1 5 2 4 3\n6\n4 2 5 3 6 1\n1\n1\n'],
      dtype=object)

In [ ]:
train_tests.filter(pl.col("id") == 4).select(pl.col("output"))[0].to_numpy().squeeze(1)

array(['4 3 2 1\n5 2 4 3 1\n6 1 5 3 4 2\n1\n'], dtype=object)